In [8]:
import pandas as pd
import numpy as np
import glob
from keras_helper import *
import kaggle_util

In [9]:
test = pd.read_csv('../input/test_simplified.csv')
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}

In [10]:

# # 0.938
# candidates = [
#     ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.18),
#     ('xception71_time', 0.927, '../result/pre_xception71_time_lw6_balance_0_adam.npy', 0.18),
# #     ('xception71_parts', 0.926, '../result/pre_xception71_parts_lw6_balance_0_adam_color.npy', 0.18),
#     ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.26),
#     ('dense64', 0.919, '../result/pre_densenet64_parts_try2_lw6_balance_0_adam_color.npy', 0.1),
# #     ('xception71', 0.918, '../result/pre_xception_raw_deep71.npy', 0.1),
# ]

# # 0.940
# candidates = [
#     ('senext50_raw', 0.930, '../result/pre_se_resnext50_raw_all_71.npy', 0.2),
#     ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.18),
#     ('xception71_time', 0.927, '../result/pre_xception71_time_lw6_balance_0_adam.npy', 0.18),
# #     ('xception71_parts', 0.926, '../result/pre_xception71_parts_lw6_balance_0_adam_color.npy', 0.18),
#     ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.26),
#     ('dense64', 0.919, '../result/pre_densenet64_parts_try2_lw6_balance_0_adam_color.npy', 0.1),
# #     ('xception71', 0.918, '../result/pre_xception_raw_deep71.npy', 0.1),
# ]

# # 0.941
# candidates = [
#     ('senext50_raw2', 0.934, '../result/pre_se_resnext50_raw_all2_71.npy', 0.2),
#     ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.18),
#     ('xception71_time', 0.927, '../result/pre_xception71_time_lw6_balance_0_adam.npy', 0.18),
# #     ('xception71_parts', 0.926, '../result/pre_xception71_parts_lw6_balance_0_adam_color.npy', 0.18),
#     ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.26),
#     ('dense64', 0.919, '../result/pre_densenet64_parts_try2_lw6_balance_0_adam_color.npy', 0.1),
# #     ('xception71', 0.918, '../result/pre_xception_raw_deep71.npy', 0.1),
# ]

# # 0.941
# candidates = [
#     ('senext50_raw2', 0.934, '../result/pre_se_resnext50_raw_all2_71.npy', 0.2),
#     ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.18),
# #     ('xception71_time', 0.927, '../result/pre_xception71_time_lw6_balance_0_adam.npy', 0.18),
# #     ('xception71_parts', 0.926, '../result/pre_xception71_parts_lw6_balance_0_adam_color.npy', 0.18),
#     ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.26),
# #     ('dense64', 0.919, '../result/pre_densenet64_parts_try2_lw6_balance_0_adam_color.npy', 0.1),
# #     ('xception71', 0.918, '../result/pre_xception_raw_deep71.npy', 0.1),
# ]

# # 0.943
# candidates = [
# #     ('senext50_raw2', 0.934, '../result/pre_se_resnext50_raw_all2_71.npy', 0.2),
#     ('senext50_raw2', 0.936, '../result/pre_se_resnext50_raw_all71_recg2.npy', 0.25),
#     ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.2),
#     ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.20),
# ]

# ??
candidates = [
    ('senext50_raw2', 0.936, '../result/pre_senext_8410.npy', 0.25),
    ('senext50_parts', 0.928,  '../result/pre_se_resnext50_parts_all_71.npy', 0.2),
    ('lstm384', 0.922, '../result/pre_lstm_conv64_units384_layer3_dropout0.2_feats9_noconvdrop2.npy', 0.20),
]

In [11]:
df = pd.DataFrame()
mix = None
for idx, (name, score, path, weight) in enumerate(candidates):
    pred = np.load(path)
    
    if mix is None:
        mix = pred * weight
    else:
        mix += pred * weight
    df[name] = pred.flatten()

In [12]:
df.corr()

,senext50_raw2,senext50_parts,lstm384
senext50_raw2,1.000000,0.975171,0.960899
senext50_parts,0.975171,1.000000,0.963823
lstm384,0.960899,0.963823,1.000000


In [13]:
top3 = preds2catids(mix)
top3cats = top3.replace(id2cat)
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]

In [14]:
kaggle_util.save_result(submission,  
                            '../result/ensemble_localbest.csv', 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/ensemble_localbest.csv.7z -m "submit"
